## Quantitative value strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports

In [114]:
import pandas as pd
import numpy as np
import requests
import xlsxwriter
import math
from scipy import stats
from scipy.stats import percentileofscore as score
from statistics import mean

## Importing Our List of Stocks & API Token

In [115]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [116]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

## Parsing Our API Call

In [117]:
price = data['latestPrice']
pe_ratio=data['peRatio']

## Executing A Batch API Call & Building Our DataFrame

In [118]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [119]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
tickers = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

my_columns = ['Tickers', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_url_call = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url_call).json()
    for symbol in symbol_string.split(','):
        try:
            d = dict(zip(my_columns, [symbol,
                                     data[symbol]['quote']['latestPrice'],
                                     data[symbol]['quote']['peRatio'],
                                     'N/A']))
            tickers.append(d)
        except:
            continue
            
final_dataframe = pd.DataFrame(tickers)
final_dataframe

,Tickers,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,141.14,35.11,N/A
1,AAL,17.48,-5.50,N/A
2,AAP,224.59,22.93,N/A
3,AAPL,176.98,29.99,N/A
4,ABBV,165.06,25.94,N/A
...,...,...,...,...
497,YUM,123.19,23.89,N/A
498,ZBH,128.07,67.13,N/A
499,ZBRA,437.52,27.31,N/A
500,ZION,73.56,7.17,N/A


## Removing Glamour Stocks
The opposite of a "value stock" is a "glamour stock".

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [120]:
final_dataframe = final_dataframe.sort_values('Price-to-Earnings Ratio',ascending=False)[:50]
final_dataframe.reset_index(drop = True, inplace = True)

## Calculating the Number of Shares to Buy

In [121]:
def portofolio_input():
    global portofolio_size, val
    portofolio_size = input('Enter the value of your portofolio: ')
    
    try:
        val = float(portofolio_size)
    except:
        print('Please enter a number')
        portofolio_size = input('Enter the value of your portofolio: ')
portofolio_input()

Enter the value of your portofolio: 
Please enter a number
Enter the value of your portofolio: 10000000


In [122]:
position_size = val / len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] =math.floor(position_size/final_dataframe['Price'][i])

,Tickers,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NOW,582.39,518.51,343
1,VTR,64.70,497.91,3091
2,DXCM,486.17,317.30,411
3,KSU,293.97,288.40,680
4,FTI,8.03,269.50,24906
5,ALB,225.82,210.45,885
6,SBAC,345.41,158.16,579
7,FIS,97.62,149.45,2048
8,CRM,212.25,148.28,942
9,EQIX,711.08,132.00,281


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

- Price-to-earnings ratio
- Price-to-book ratio
- Price-to-sales ratio
- Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
- Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [123]:
symbol = 'NOW'
batch_api_url_call = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_url_call).json()

In [124]:
# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

# Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gp = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [125]:
rv_columns = ['Ticker',
             'Price',
             'Number of Shares to Buy',
             'Price-to-Earnings Ratio',
             'PE Percentile',
             'Price-to-Book Ratio',
             'PB Percentile',
             'Price-to-Sales Ratio',
             'PS Percentile',
             'EV/EBITDA',
             'EV/EBITDA Percentile',
             'EV/GP',
             'EV/GP Percentile',
             'RV Score']

rv_dataframe = pd.DataFrame(columns = rv_columns)
rv_tickers = []
for symbol_string in symbol_strings:
    batch_api_url_call = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url_call).json()
    for symbol in symbol_string.split(','):
        try:
            enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
            ebitda = data[symbol]['advanced-stats']['EBITDA']
            gross_profit = data[symbol]['advanced-stats']['grossProfit']
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        except KeyError:
            continue
        try:
            ev_to_gp = enterprise_value/gross_profit
        except TypeError:
            ev_to_gp = np.NaN
        except KeyError:
            continue
        rv_d = dict(zip(rv_columns, [symbol,
                                    data[symbol]['quote']['latestPrice'],
                                    'N/A',
                                    data[symbol]['quote']['peRatio'],
                                    'N/A',
                                    data[symbol]['advanced-stats']['priceToBook'],
                                    'N/A',
                                    data[symbol]['advanced-stats']['priceToSales'],
                                    'N/A',
                                    ev_to_ebitda,
                                    'N/A',
                                    ev_to_gp,
                                    'N/A',
                                    'N/A']))
        rv_tickers.append(rv_d)

rv_dataframe = pd.DataFrame(rv_tickers)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,140.92,N/A,35.52,N/A,8.25,N/A,6.3500,N/A,24.907457,N/A,12.033629,N/A,N/A
1,AAL,17.22,N/A,-5.80,N/A,-1.50,N/A,0.3842,N/A,-11.864407,N/A,1.215118,N/A,N/A
2,AAP,223.80,N/A,23.58,N/A,4.39,N/A,1.2000,N/A,12.678941,N/A,2.852355,N/A,N/A
3,AAPL,178.85,N/A,30.33,N/A,41.17,N/A,7.8400,N/A,23.277958,N/A,17.978103,N/A,N/A
4,ABBV,166.02,N/A,25.18,N/A,19.37,N/A,5.2300,N/A,12.343089,N/A,8.784427,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,122.06,N/A,23.72,N/A,-4.21,N/A,5.3400,N/A,20.218081,N/A,9.429021,N/A,N/A
498,ZBH,129.20,N/A,65.82,N/A,2.15,N/A,3.4800,N/A,19.249530,N/A,5.767800,N/A,N/A
499,ZBRA,444.11,N/A,28.35,N/A,7.74,N/A,4.0900,N/A,20.829872,N/A,9.003271,N/A,N/A
500,ZION,72.24,N/A,7.26,N/A,1.61,N/A,3.0800,N/A,5.524030,N/A,2.810731,N/A,N/A


## Dealing With Missing Data in Our DataFrame

In [126]:
rv_dataframe[rv_dataframe.isnull().any(axis= 1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,182.800,N/A,60.63,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,335.760,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,373.110,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.98,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,28.530,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,51.260,N/A,14.81,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,58.490,N/A,33.44,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,39.000,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
193,FRT,124.280,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2931.440,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

- Drop missing data from the data set (pandas' `dropna` method is useful here)
- Replace missing data with a new value (pandas' `fillna` method is useful here)

In [127]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio','EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)

## Calculating Value Percentiles

In [128]:
# from scipy.stats import percentileofscore as score
# The same logic as we did in quantitative momentum strategy but with different method
metrics = {'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'}
for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,140.92,N/A,35.52,74.302789,8.25,79.880478,6.3500,79.780876,24.907457,81.673307,12.033629,79.282869,N/A
1,AAL,17.22,N/A,-5.80,3.984064,-1.50,5.976096,0.3842,1.992032,-11.864407,4.780876,1.215118,3.386454,N/A
2,AAP,223.80,N/A,23.58,48.605578,4.39,60.856574,1.2000,15.737052,12.678941,42.231076,2.852355,14.541833,N/A
3,AAPL,178.85,N/A,30.33,64.741036,41.17,97.410359,7.8400,85.059761,23.277958,77.888446,17.978103,91.035857,N/A
4,ABBV,166.02,N/A,25.18,53.386454,19.37,93.227092,5.2300,72.808765,12.343089,39.641434,8.784427,57.768924,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,122.06,N/A,23.72,49.800797,-4.21,5.577689,5.3400,73.406375,20.218081,71.314741,9.429021,65.737052,N/A
498,ZBH,129.20,N/A,65.82,93.426295,2.15,31.673307,3.4800,51.494024,19.249530,67.330677,5.767800,36.653386,N/A
499,ZBRA,444.11,N/A,28.35,60.557769,7.74,78.486056,4.0900,58.964143,20.829872,72.709163,9.003271,63.545817,N/A
500,ZION,72.24,N/A,7.26,9.561753,1.61,22.011952,3.0800,46.912351,5.524030,11.553785,2.810731,13.944223,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [129]:
# from statistics import mean
for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row,'RV Score'] = mean(value_percentiles)

## Selecting the 50 Best Value Stocks

In [130]:
rv_dataframe.sort_values('RV Score', ascending=False, inplace=True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop=True, inplace = True)

## Calculating the Number of Shares to Buy
We'll use the portfolio_input function that we created earlier to accept our portfolio size.

Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [131]:
portofolio_input()

Enter the value of your portofolio: 10000000


In [132]:
position_size = val/ len(rv_dataframe.index)
for i in range(0, len(rv_dataframe.index)):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe['Price'][i])

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,NOW,573.17,348,507.42,99.800797,30.97,96.414343,19.60,98.605578,156.076832,99.800797,24.136243,96.613546,98.247012
1,DXCM,489.66,408,322.90,99.601594,21.00,94.223108,19.63,98.804781,121.587310,99.601594,27.779202,98.605578,98.167331
2,NVDA,281.32,710,73.43,94.820717,26.41,95.816733,26.17,100.0,59.851835,98.207171,39.146610,99.40239,97.649402
3,PAYC,345.64,578,103.41,96.812749,23.75,95.61753,20.39,99.40239,63.600676,98.605578,22.184108,95.219124,97.131474
4,FTNT,348.06,574,96.06,96.215139,71.28,99.003984,16.51,96.613546,72.088541,99.003984,20.431165,93.824701,96.932271
5,IDXX,558.18,358,64.59,93.227092,67.60,98.804781,14.74,95.219124,44.422348,96.015936,24.917372,97.410359,96.135458
6,CDNS,166.46,1201,63.67,92.828685,16.07,91.633466,15.35,95.61753,52.754423,97.609562,16.201195,88.645418,93.266932
7,ISRG,295.82,676,63.61,92.629482,8.75,81.474104,18.70,97.410359,46.842863,97.011952,24.830129,97.011952,93.10757
8,TYL,437.95,456,115.91,97.410359,7.72,78.286853,11.27,92.031873,84.518632,99.203187,26.361400,97.808765,92.948207
9,ABMD,321.05,622,112.26,97.211155,10.61,83.266932,14.67,95.01992,51.299853,97.410359,17.545062,90.438247,92.669323


## Formatting and Saving Our Excel Output

In [136]:
writer = pd.ExcelWriter('value_startegy.xlsx', engine= 'xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index= False)
writer.save()